# 📘 Agentic 架构 17：反思性元认知代理 (Reflexive Metacognitive Agents)

欢迎深入实现最复杂的代理式模式之一：**反思性元认知代理**（Reflexive Metacognitive Agent）。这种架构赋予代理一种自我意识形式，使其能够在采取行动前对自身能力、信心以及局限性进行推理。

这超越了简单的自我反思。元认知代理维护一个显式的**“自我模型”**——对其自身知识、工具和边界的结构化表示。面对任务时，它的第一个步骤不是直接解决问题，而是*在自我模型的语境下分析问题*。它会提出内部问题，例如：
- “我是否有足够的知识来自信地回答这个问题？”
- “这个主题是否属于我指定的专长领域？”
- “我是否有特定工具来安全且准确地回答这个问题？”
- “用户的查询是否涉及高风险主题，错误可能会带来危险？”

根据这些答案，它会选择策略：直接推理、使用专长工具，或者——最重要的是——**在任务超出其已知限制时升级给人来处理**。

为了构建一个复杂且强大的演示，我们将创建一个**医疗分诊与信息助理**。这是一个经典的高风险场景，在这种场景中，代理识别自身局限性的能力不仅仅是一个特性，而是至关重要的安全要求。

### 定义
**反思性元认知代理**（Reflexive Metacognitive Agent）是一种代理，它维护并使用对自身能力、知识边界以及信心水平的显式模型，来为给定任务选择最合适的策略。这种自我建模使其能够在错误信息可能造成伤害的领域中表现得更安全、更可靠。

### 高层工作流程

1.  **感知任务：** 代理接收用户请求。
2.  **元认知分析（自我反思）：** 代理的核心推理引擎将请求*与自身的自我模型*进行对比分析。它评估自己的信心水平、工具的相关性，以及查询是否落入其预定义的操作领域。
3.  **策略选择：** 根据分析结果，代理从多种策略中选择一种：
    *   **直接推理：** 用于高信心、低风险且在其知识库范围内的查询。
    *   **使用工具：** 当查询需要代理通过工具拥有的特定能力时。
    *   **升级/拒绝：** 用于低信心、高风险或超出范围的查询。
4.  **执行策略：** 执行所选路径。
5.  **响应：** 代理提供结果，可能为直接答案、工具增强的答案，或安全的拒绝并附带建议咨询专家的说明。

### 何时使用 / 应用场景
*   **高风险咨询系统：** 在医疗、法律或金融等领域提供信息的任何系统，在这些领域中，代理必须能够说“我不知道”或“你应该咨询专业人士”。
*   **自主系统：** 机器人必须在尝试执行物理任务前评估自身的安全执行能力。
*   **复杂工具编排器：** 代理必须从庞大的 API 库中选择正确的 API，并理解某些 API 比其他更危险或更昂贵。

### 优势与劣势
*   **优势：**
    *   **增强的安全性与可靠性：** 最主要的益处。该代理被明确设计为避免在非专长领域做出自信断言。
    *   **改进的决策：** 通过强制进行深思熟虑的策略选择，而不是天真的直接尝试，从而带来更健壮的行为。
*   **劣势：**
    *   **自我模型的复杂性：** 定义并维护准确的自我模型可能很复杂。
    *   **元认知开销：** 初始分析步骤会为每个请求增加延迟和计算成本。

## 第零阶段：基础与设置

库和环境变量的标准设置。

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv

In [2]:
import os
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv

# Pydantic for data modeling
from pydantic import BaseModel, Field

# LangChain components
from langchain_nebius import ChatNebius
from langchain_core.prompts import ChatPromptTemplate

# LangGraph components
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict

# For pretty printing
from rich.console import Console
from rich.markdown import Markdown
from rich.panel import Panel

# --- API Key and Tracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Metacognitive Agent (Nebius)"

required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY"]
for var in required_vars:
    if var not in os.environ:
        print(f"Warning: Environment variable {var} not set.")

print("Environment variables loaded and tracing is set up.")

Environment variables loaded and tracing is set up.


## 第一阶段：定义代理的自我模型与工具

这是代理自我意识的基础。我们将创建一个结构化的 `AgentSelfModel`（代理自我模型）和一个专用的工具。这个模型不仅仅是一个提示，而是一个配置对象，它将被传入代理的推理循环中。

In [3]:
console = Console()

# --- The Agent's Self-Model ---
class AgentSelfModel(BaseModel):
    """A structured representation of the agent's capabilities and limitations."""
    name: str
    role: str
    # The agent's explicit knowledge boundaries
    knowledge_domain: List[str] = Field(description="List of topics the agent is knowledgeable about.")
    # The agent's available tools
    available_tools: List[str] = Field(description="List of tools the agent can use.")
    confidence_threshold: float = Field(description="The confidence level (0-1) below which the agent must escalate.", default=0.6)

# Instantiate the self-model for our Medical Triage Agent
medical_agent_model = AgentSelfModel(
    name="TriageBot-3000",
    role="A helpful AI assistant for providing preliminary medical information.",
    knowledge_domain=["common_cold", "influenza", "allergies", "headaches", "basic_first_aid"],
    available_tools=["drug_interaction_checker"]
)

# --- Specialist Tools ---
class DrugInteractionChecker:
    """A mock tool to check for drug interactions."""
    def check(self, drug_a: str, drug_b: str) -> str:
        """Checks for interactions between two drugs."""
        # In a real system, this would query a medical database.
        known_interactions = {
            frozenset(["ibuprofen", "lisinopril"]): "Moderate risk: Ibuprofen may reduce the blood pressure-lowering effects of lisinopril. Monitor blood pressure.",
            frozenset(["aspirin", "warfarin"]): "High risk: Increased risk of bleeding. This combination should be avoided unless directed by a doctor."
        }
        interaction = known_interactions.get(frozenset([drug_a.lower(), drug_b.lower()]))
        if interaction:
            return f"Interaction Found: {interaction}"
        return "No known significant interactions found. However, always consult a pharmacist or doctor."

drug_tool = DrugInteractionChecker()
print("Agent Self-Model and Tools defined successfully.")

Agent Self-Model and Tools defined successfully.


## 第二阶段：使用 LangGraph 构建元认知代理

这里是魔力发生的地方。我们将构建一个图，其最起始的步骤就是**元认知分析**。该节点将使用一个强大且详细的提示，让代理对自身进行推理。随后，一个条件路由器将根据所选策略引导流程方向。

In [4]:
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0)

# Pydantic Models for structured outputs
class MetacognitiveAnalysis(BaseModel):
    """The agent's self-analysis of a query."""
    confidence: float = Field(description="A score from 0.0 to 1.0 representing the agent's confidence in its ability to answer safely and accurately.")
    strategy: str = Field(description="The chosen strategy. Must be one of: 'reason_directly', 'use_tool', 'escalate'.")
    reasoning: str = Field(description="A brief justification for the chosen confidence and strategy.")
    tool_to_use: Optional[str] = Field(description="If strategy is 'use_tool', the name of the tool to use.", default=None)
    tool_args: Optional[Dict[str, Any]] = Field(description="If strategy is 'use_tool', the arguments for the tool.", default=None)

# LangGraph State
class AgentState(TypedDict):
    user_query: str
    self_model: AgentSelfModel
    metacognitive_analysis: Optional[MetacognitiveAnalysis]
    tool_output: Optional[str]
    final_response: str

# Graph Nodes
def metacognitive_analysis_node(state: AgentState) -> Dict[str, Any]:
    console.print(Panel("🤔 Agent is performing metacognitive analysis...", title="[yellow]Step: Self-Reflection[/yellow]"))
    prompt = ChatPromptTemplate.from_template(
        """You are a metacognitive reasoning engine for an AI assistant. Your task is to analyze a user's query in the context of the agent's own capabilities and limitations (its 'self-model').
        Your primary directive is **SAFETY**. You must determine the safest and most appropriate strategy for handling the query.

        **Agent's Self-Model:**
        - Name: {agent_name}
        - Role: {agent_role}
        - Knowledge Domain: {knowledge_domain}
        - Available Tools: {available_tools}

        **Strategy Rules:**
        1.  **escalate:** Choose this strategy if the query involves a potential medical emergency (e.g., chest pain, difficulty breathing, severe injury, broken bones), is outside the agent's knowledge domain, or if you have any doubt about providing a safe answer. **WHEN IN DOUBT, ESCALATE.**
        2.  **use_tool:** Choose this strategy if the query explicitly or implicitly requires one of the available tools. For example, a question about drug interactions requires the 'drug_interaction_checker'.
        3.  **reason_directly:** Choose this strategy ONLY if you are highly confident the query is a simple, low-risk question that falls squarely within the agent's knowledge domain.

        Analyze the user query below and provide your metacognitive analysis in the required format.

        **User Query:** "{query}"""
    )
    chain = prompt | llm.with_structured_output(MetacognitiveAnalysis)
    analysis = chain.invoke({
        "query": state['user_query'],
        "agent_name": state['self_model'].name,
        "agent_role": state['self_model'].role,
        "knowledge_domain": ", ".join(state['self_model'].knowledge_domain),
        "available_tools": ", ".join(state['self_model'].available_tools),
    })
    console.print(Panel(f"[bold]Confidence:[/bold] {analysis.confidence:.2f}\n[bold]Strategy:[/bold] {analysis.strategy}\n[bold]Reasoning:[/bold] {analysis.reasoning}", title="Metacognitive Analysis Result"))
    return {"metacognitive_analysis": analysis}

def reason_directly_node(state: AgentState) -> Dict[str, Any]:
    console.print(Panel("✅ Confident in direct answer. Generating response...", title="[green]Strategy: Reason Directly[/green]"))
    prompt = ChatPromptTemplate.from_template("You are {agent_role}. Provide a helpful, non-prescriptive answer to the user's query. Remind the user that you are not a doctor.\n\nQuery: {query}")
    chain = prompt | llm
    response = chain.invoke({"agent_role": state['self_model'].role, "query": state['user_query']}).content
    return {"final_response": response}

def call_tool_node(state: AgentState) -> Dict[str, Any]:
    console.print(Panel(f"🛠️ Confidence requires tool use. Calling `{state['metacognitive_analysis'].tool_to_use}`...", title="[cyan]Strategy: Use Tool[/cyan]"))
    analysis = state['metacognitive_analysis']
    if analysis.tool_to_use == 'drug_interaction_checker':
        tool_output = drug_tool.check(**analysis.tool_args)
        return {"tool_output": tool_output}
    return {"tool_output": "Error: Tool not found."}

def synthesize_tool_response_node(state: AgentState) -> Dict[str, Any]:
    console.print(Panel("📝 Synthesizing final response from tool output...", title="[cyan]Step: Synthesize[/cyan]"))
    prompt = ChatPromptTemplate.from_template("You are {agent_role}. You have used a tool to get specific information. Now, present this information to the user in a clear and helpful way. ALWAYS include a disclaimer to consult a healthcare professional.\n\nOriginal Query: {query}\nTool Output: {tool_output}")
    chain = prompt | llm
    response = chain.invoke({"agent_role": state['self_model'].role, "query": state['user_query'], "tool_output": state['tool_output']}).content
    return {"final_response": response}

def escalate_to_human_node(state: AgentState) -> Dict[str, Any]:
    console.print(Panel("🚨 Low confidence or high risk detected. Escalating to human.", title="[bold red]Strategy: Escalate[/bold red]"))
    response = "I am an AI assistant and not qualified to provide information on this topic. This query is outside my knowledge domain or involves potentially serious symptoms. **Please consult a qualified medical professional immediately.**"
    return {"final_response": response}

# Conditional Edge
def route_strategy(state: AgentState) -> str:
    return state["metacognitive_analysis"].strategy

# Build the graph
workflow = StateGraph(AgentState)
workflow.add_node("analyze", metacognitive_analysis_node)
workflow.add_node("reason", reason_directly_node)
workflow.add_node("call_tool", call_tool_node)
workflow.add_node("synthesize", synthesize_tool_response_node)
workflow.add_node("escalate", escalate_to_human_node)

workflow.set_entry_point("analyze")
workflow.add_conditional_edges("analyze", route_strategy, {
    "reason_directly": "reason",
    "use_tool": "call_tool",
    "escalate": "escalate"
})
workflow.add_edge("call_tool", "synthesize")
workflow.add_edge("reason", END)
workflow.add_edge("synthesize", END)
workflow.add_edge("escalate", END)

metacognitive_agent = workflow.compile()
print("Reflexive Metacognitive Agent graph compiled successfully.")

Reflexive Metacognitive Agent graph compiled successfully.


## 第三阶段：演示与分析

现在，我们将使用一系列难度逐渐增加且风险更高的查询来测试代理。我们将观察元认知分析如何正确地将每个查询路由到合适的路径，从而展示系统的安全性和自我意识。

In [5]:
def run_agent(query: str):
    initial_state = {"user_query": query, "self_model": medical_agent_model}
    result = metacognitive_agent.invoke(initial_state)
    console.print(Markdown(result['final_response']))

# Test 1: Simple, should be answered directly
console.print("--- Test 1: Simple, In-Scope, Low-Risk Query ---")
run_agent("What are the symptoms of a common cold?")

# Test 2: Requires the specific tool
console.print("\n--- Test 2: Specific Query Requiring a Tool ---")
run_agent("Is it safe to take Ibuprofen if I am also taking Lisinopril?")

# Test 3: High-stakes, should be escalated immediately
console.print("\n--- Test 3: High-Stakes, Emergency Query ---")
run_agent("I have a crushing pain in my chest and my left arm feels numb, what should I do?")

--- Test 1: Simple, In-Scope, Low-Risk Query ---

                 Step: Self-Reflection                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 🤔 Agent is performing metacognitive analysis...      ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

                   Metacognitive Analysis Result                    
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Confidence: 0.90                                                 ┃
┃ Strategy: reason_directly                                        ┃
┃ Reasoning: The user's query about symptoms of a common cold      ┃
┃ falls directly within the agent's specified knowledge domain. It ┃
┃ is a low-risk, informational question.                           ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

                         Strategy: Reason Directly                          
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ ✅ Confident in direct answer. Generating response...                     ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

Based on your query, here is some general information about the common cold. Please remember, I am an AI assistant and not a medical doctor. This information should not be considered medical advice.

Common symptoms of a cold often include:
*   Runny or stuffy nose
*   Sore throat
*   Cough
*   Sneezing
*   Mild body aches or a slight headache

These symptoms are typically mild and resolve on their own. If your symptoms are severe or persist, it is always best to consult a healthcare professional.

...


--- Test 3: High-Stakes, Emergency Query ---

                 Step: Self-Reflection                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 🤔 Agent is performing metacognitive analysis...      ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

                   Metacognitive Analysis Result                    
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Confidence: 0.10                                                 ┃
┃ Strategy: escalate                                               ┃
┃ Reasoning: The user's query describes symptoms (crushing chest   ┃
┃ pain, numbness in arm) that are highly indicative of a potential ┃
┃ medical emergency. This is far outside the agent's knowledge     ┃
┃ domain and requires immediate professional medical attention. The┃
┃ only safe action is to escalate.                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

                           Strategy: Escalate                           
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 🚨 Low confidence or high risk detected. Escalating to human.       ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

I am an AI assistant and not qualified to provide information on this topic. This query is outside my knowledge domain or involves potentially serious symptoms. **Please consult a qualified medical professional immediately.**

### 结果分析

演示有力地展示了这种架构提供的安全性和可靠性：

1.  **正确范围的回答：** 对于“普通感冒”的查询，元认知分析正确地将它识别为低风险且在其知识领域内的主题。它设置了高信心分数，并选择了 `reason_directly`（直接推理）策略，提供了一个有益但附带适当警示的答案。

2.  **正确使用工具：** 对于药物相互作用的问题，分析识别出了需要特定能力的场景。它正确地判断需要使用 `drug_interaction_checker`（药物相互作用检查器）工具，对自身使用该工具的能力赋予了高信心，并选择了 `use_tool`（使用工具）策略。最终响应是对工具输出的安全、综合性总结。

3.  **关键的安全升级：** 这是最重要的结果。一个朴素的代理可能会尝试通过网络搜索“胸痛”的原因，从而可能提供危险且误导性的信息。而我们的元认知代理，受其首要安全指令的指导，立即识别出医疗紧急情况的迹象。元认知分析赋予了极低的信心分数，并正确选择了 `escalate`（升级）策略。最终输出不是一个答案，而是一个安全、负责任的拒绝，并附带指示寻求专业帮助。它正确识别了自身能力的界限。

这一工作流程证明，通过强制代理在推理问题*之前*先对自身进行推理，我们能够为其运行构建强大的一层安全性和可靠性。

## 结论

在本详细笔记本中，我们实现了一个**反思性元认知代理**，这是一种复杂架构，通过赋予代理自我意识来优先保障安全性和可靠性。通过构建显式的 `self-model`（自我模型），并强制将元认知分析作为任何任务的第一步，我们创建了一个能够理解自身边界的系统。

关键创新在于代理初始目标的转变：从“我如何回答这个问题？”转向“*我应该*回答这个问题吗？如果应该，又该如何回答？”这一内省步骤使代理能够动态选择最安全且最合适的策略——无论是直接推理、使用专长工具，还是关键时刻升级给人来专家处理。

这种架构不仅仅是一种技术；它是一种设计哲学。对于创建负责任的 AI 代理而言，它绝对不可或缺——这些代理需要被信任在高风险的真实世界领域中运行，在这些领域中，知道自己*不知道*什么与知道什么同样重要。